In [53]:
import cv2
import numpy as np

In [54]:
path = './test.jpg'
img = cv2.imread(path, 0)

In [55]:
def display(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Image Negative

In [56]:
negative = 255 - img
display("Negative", negative)

### Log transform
Essentially brightens a dark image.

In [57]:
# select a suitable constant c
max_pixel = float(np.max(img))
c = 255 / (np.log(1 + float(max_pixel)))  

# apply log transform
log_transformed = c * np.log(1 + img)  

# convert float to int
log_transformed = log_transformed.astype(np.uint8) 
 
display("Log transformed", log_transformed)


/var/folders/41/v7x6qrn91r5ccc86q_ps3c7c0000gn/T/ipykernel_54331/2377519716.py:6: RuntimeWarning: divide by zero encountered in log
  log_transformed = c * np.log(1 + img)
/var/folders/41/v7x6qrn91r5ccc86q_ps3c7c0000gn/T/ipykernel_54331/2377519716.py:9: RuntimeWarning: invalid value encountered in cast
  log_transformed = log_transformed.astype(np.uint8)


### Power law (gamma) transform

$ s = c r ^ \gamma $, where $c$ and $\gamma$ are positive constants. 

A variety of transformations can be obtained by simply varying $\gamma$. Curves generated with $\gamma < 1$ have the opposite effect to those generated with $\gamma > 1$.

In [58]:
gs = [0.25, 0.5, 1, 1.5, 2]
for gamma in gs:
    gamma_corrected = 255 * (img / 255) ** gamma
    gamma_corrected = gamma_corrected.astype(np.uint8)
    display(f"gamma = {gamma}", gamma_corrected)

## Piecewise-linear functions

### Contrast stretching

In [61]:
def F(pixel, r1, s1, r2, s2):
    if 0 <= pixel < r1:
        return (s1 / r1) * pixel
    elif r1 <= pixel < r2:
        return ((s2 - s1) / (r2 - r1)) * (pixel - r1) + s1
    else:
        return ((255 - s2) / (255 - r2)) * (pixel - r2) + s2
    
F_vec = np.vectorize(F)
constrast_stretched = F_vec(img, 70, 0, 140, 255)
display("High constast", constrast_stretched)

### Intensity-level slicing
Also known as gray-level slicing

In [62]:
def F(pixel, a, b):
    if a <= pixel <= b:
        return 255
    return 0

F_vec = np.vectorize(F)
intensity_slice = F_vec(img, 50, 150).astype(np.uint8)
display("Intensity slice", intensity_slice)